In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model

In [2]:
model = YOLO("yolov8n.pt")

# Initialize video capture

In [3]:
video_path = 'D:/Internship Assignments/Auriga Tech Solutions Private Limited/Traffic/traffic_vid.mp4'
cap = cv2.VideoCapture(video_path)

# Define stop line position (modify based on your video)

In [8]:
original_stop_line_position = 500 

In [9]:
display_width, display_height = 640, 480

# Initialize car counter

In [10]:
car_count = 0
detected_cars = []

# Function to check if a car is crossing the stop line

In [11]:
def is_crossing_stop_line(y1, y2, stop_line_position):
    return y1 < stop_line_position < y2

In [12]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get original frame dimensions
    original_height, original_width, _ = frame.shape

    # Calculate the scaling factor
    scale_x = display_width / original_width
    scale_y = display_height / original_height

    # Scale the stop line position
    stop_line_position = int(original_stop_line_position * scale_y)

    # Run YOLOv8 on the frame
    results = model(frame)

    # Resize the frame for display
    frame_resized = cv2.resize(frame, (display_width, display_height))

    # Extract bounding boxes and class labels from YOLO results
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])
        cls = int(result.cls)
        if cls == 2:  # Class 2 corresponds to 'car' in COCO dataset
            car_id = f'{x1}_{y1}_{x2}_{y2}'
            if is_crossing_stop_line(y1, y2, stop_line_position):
                if car_id not in detected_cars:
                    detected_cars.append(car_id)
                    car_count += 1
                # Draw bounding box in a distinct color for cars crossing the stop line
                cv2.rectangle(frame_resized, (int(x1*scale_x), int(y1*scale_y)), (int(x2*scale_x), int(y2*scale_y)), (0, 0, 255), 2)
                cv2.putText(frame_resized, 'Car (counted)', (int(x1*scale_x), int(y1*scale_y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            else:
                # Draw bounding box for other cars
                cv2.rectangle(frame_resized, (int(x1*scale_x), int(y1*scale_y)), (int(x2*scale_x), int(y2*scale_y)), (0, 255, 0), 2)
                cv2.putText(frame_resized, 'Car', (int(x1*scale_x), int(y1*scale_y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Draw the stop line
    cv2.line(frame_resized, (0, stop_line_position), (display_width, stop_line_position), (255, 0, 0), 2)

    # Display car count on the resized frame
    cv2.putText(frame_resized, f'Car count: {car_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the resized frame
    cv2.imshow('Frame (Resized)', frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


0: 384x640 8 cars, 8 traffic lights, 389.9ms
Speed: 16.5ms preprocess, 389.9ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 5 traffic lights, 300.8ms
Speed: 8.6ms preprocess, 300.8ms inference, 16.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 6 traffic lights, 281.9ms
Speed: 0.0ms preprocess, 281.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 5 traffic lights, 233.1ms
Speed: 0.0ms preprocess, 233.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 5 traffic lights, 265.9ms
Speed: 0.0ms preprocess, 265.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 4 traffic lights, 249.3ms
Speed: 0.0ms preprocess, 249.3ms inference, 16.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 4 traffic lights, 249.9ms
Speed: 15.6ms preprocess, 249.9ms inference, 0.0ms postprocess per image at shape (1, 3,

KeyboardInterrupt: 

In [10]:
cap.release()
cv2.destroyAllWindows()

In [11]:
print(f'Total car count: {car_count}')

Total car count: 0
